In [1]:
import os
import pandas as pd
from math import ceil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
import datetime
#pd.show_versions(as_json=False)

threshold = 1000

In [2]:
# Load Data
def load_raw(fn):
    datapath = os.path.join('../csse_covid_19_data/csse_covid_19_time_series/', fn)

    #datapath = '../csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    
    raw = pd.read_csv(datapath)
    #del raw['Lat']
    #del raw['Long']
    #raw= raw.replace('Mainland China','China')
    raw = raw[raw.iso2=='US']
    raw = raw[pd.notnull(raw.UID) & ~(raw.Lat==0.0)]
    # fix HI/AK later
    raw = raw[~(raw.Province_State=='Alaska') & ~(raw.Province_State=='Hawaii')]
    return raw

raw = load_raw('time_series_covid19_confirmed_us.csv')
#raw_deaths = load_raw('time_series_covid19_deaths_global.csv')

raw


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20
5,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,43,44,42,45,48,53,53,58,61,67
6,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,174,174,175,181,187,188,189,196,205,208
7,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,37,39,42,43,45,45,47,47,51,53
8,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,42,42,42,42,43,42,43,43,44,44
9,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,36,37,39,40,40,40,40,42,44,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3256,84070016,US,USA,840,NaN,Central Utah,Utah,US,39.372319,-111.575868,...,22,23,23,23,24,25,25,26,27,27
3257,84070017,US,USA,840,NaN,Southeast Utah,Utah,US,38.996171,-110.701396,...,14,12,13,15,11,11,12,12,12,13
3258,84070018,US,USA,840,NaN,Southwest Utah,Utah,US,37.854472,-111.441876,...,93,98,101,103,111,119,121,128,132,138
3259,84070019,US,USA,840,NaN,TriCounty,Utah,US,40.124915,-109.517442,...,11,13,13,13,14,14,14,14,14,14


In [4]:
%%capture
%matplotlib inline
from math import log
plt.rcParams["figure.figsize"] = [10, 5]
x=raw['Long_']
y=raw['Lat']
#c=raw['4/19/20'].apply(lambda x: log(x) if x>0 else 0)
weeks = 12
window = 7
dates=raw.columns[-weeks*7:]
#print(dates)
new = [(raw[dates[i]]-raw[dates[i-window]]).apply(lambda x: log(x) if x>0 else 0) for i in range(window,len(dates))]
#print(c.min(),c.max())
plt.gca().set_aspect('equal', adjustable='box')
fig, ax = plt.subplots()

def plot_map(date_tuple):
    c,dt = date_tuple
    #print(dt)
    ax.clear()
    ax.scatter(x,y, c=c, alpha=0.5, cmap='autumn_r')
    ax.set_yticks([])
    ax.set_xticks([])
    plt.box(False)
    ax.text(0.95, 0.1, dt, transform=ax.transAxes, color='#222222', size=24, ha='right', weight=800)

frames = list(zip(new,dates[window:]))
animator = animation.FuncAnimation(fig, plot_map, frames=frames, repeat=False)
#HTML(animator.to_jshtml()) 

maxDate=raw.columns[-1].replace('/','-')
#animator.save('data/animation-{}.gif'.format(maxDate))
animator.save('data/animation-latest.gif')
